In [1]:
import torch
from torch_geometric.data import Data
from tqdm import tqdm

# Creating Edges
Index string_db, and connect nodes with a combined score over 500 (arbitrary)

In [2]:
import json

stringdb = json.load(open("../Corpuses/string_db.json"))

Mapping all gene names to a vocab, then creating an edge connection tensor

In [3]:
edge_index = [[], []] # starts, ends
vocab = {}
idx = 0
for k, v in tqdm(stringdb.items()):
    if 'combined_score' in v and int(v['combined_score']) > 500:
        if k not in vocab:
            vocab[k] = idx
            idx += 1
        if v['protein2'] not in vocab:
            vocab[v['protein2']] = idx
            idx += 1
        
        edge_index[0].extend([vocab[k], vocab[v['protein2']]])
        edge_index[1].extend([vocab[v['protein2']], vocab[k]])
edge_index = torch.tensor(edge_index, dtype=torch.long)
print(edge_index.shape)

100%|██████████| 19386/19386 [00:00<00:00, 1899740.13it/s]

torch.Size([2, 3702])


# Creating Node Feature Vectors

In [4]:
import sys
import torch
from bioservices import UniProt
sys.path.append("../")
from protbert.inference import ProtBertModule

protbert = ProtBertModule(torch.device('cpu'))
u = UniProt(verbose=False)

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
WARNING [bioservices:UniProt:119]:  The URL (https://www.uniprot.org) provided cannot be reached.


In [5]:
from pprint import pprint
from tqdm import tqdm

feat_vecs = [] # attach in protbert inference here

for node in tqdm(vocab.keys()):
    # Finding associated protein for each gene
    o = u.search(node, frmt="fasta").split("\n")
    fasta = "".join(o[1:])
    
    # inference
    fasta = protbert.encode(fasta)
    preds = protbert(fasta).logits
    preds = torch.squeeze(preds).cpu().detach().numpy()
    feat_vecs.append(preds)
    del preds
    del fasta

  0%|          | 1/3289 [00:02<1:56:57,  2.13s/it]

In [ ]:
data = Data(x=feat_vecs, edge_index=edge_index)

Error: Session cannot generate requests